In [ ]:
import requests
import os
import math
import json
import pprint as pp
from dotenv import load_dotenv
 
load_dotenv()

def read_input():
    # read input file
    with open('inputs.json', 'r') as inputs:
        data=inputs.read()

    # parse input file
    input_json = json.loads(data)

    # seting main parameters
    input_json['API_KEY'] = os.environ["API_KEY_MAPS"]
    input_json['API_ENDPOINT_URL'] = "https://maps.googleapis.com/maps/api/staticmap"
    input_json['IMAGE_DIR'] = "C:/Users/fonss/Documents/Programacion/GIT/sid_image_extractor/maps_images/"
    return input_json

input_json = read_input()

In [ ]:
# Generate the name of the image depending of the input data
def get_image_name(data, input_json):

    name =  data['name']
    zoom = str(input_json['image_params']['zoom'])
    size = str(input_json['image_params']['size'])
    pos = str(data['center']['pos'])

    image_ref = name + '_' + zoom + '_' + size + '_' + pos + '.png'
    return image_ref

# Generate the folder name for the data input 
def get_folder_ref(data, input_json):

    name =  data['name']
    grid_size = str(data['grid_size'])
    zoom = str(input_json['image_params']['zoom'])
    size = str(input_json['image_params']['size'])

    folder_ref = input_json['IMAGE_DIR'] + name + '_' + grid_size + '_' + zoom + '_' + size + '/'
    return folder_ref

#Create the folder and saves the image
def save_image(data, input_json, img):

    folder_ref = get_folder_ref( data , input_json )

    if not os.path.exists(folder_ref):
        os.makedirs(folder_ref)

    image_name = get_image_name( data , input_json )


    image_ref = folder_ref + image_name
    # save image in directory
    
    with open(image_ref, 'wb') as f:
        f.write(img)
    

def get_image(data, input_json):

    params = {
        'center': str(data['center']['lat']) + ',' + str(data['center']['lon']),
        'zoom': input_json['image_params']['zoom'],
        'maptype': input_json['image_params']['maptype'],
        'size': str(input_json['image_params']['size']) + 'x' + str(input_json['image_params']['size']),
        'scale':input_json['image_params']['scale'],
        'key': input_json['API_KEY'],
    }

    res = requests.get(url = input_json['API_ENDPOINT_URL'], params = params)
    return res.content

def get_m_per_px(data, input_json):
    m_per_px_lat = 156543.03392 * math.cos(data['center']['lat'] * math.pi / 180) / math.pow(2, input_json['image_params']['zoom'])
    m_per_px_lon = 156543.03392 * math.cos(data['center']['lon'] * math.pi / 180) / math.pow(2, input_json['image_params']['zoom'])
    return m_per_px_lat, m_per_px_lon

def get_deg_per_m(data):
    deg_per_m_lat = 1 / 111320
    deg_per_m_lon = -1 / (40075000 * math.cos(data['center']['lat']) / 360)
    return deg_per_m_lat, deg_per_m_lon


def get_grid(data, input_json):
    m_per_px_lat , m_per_px_lon = get_m_per_px(data,input_json)
    deg_per_m_lat , deg_per_m_lon = get_deg_per_m(data)


    overlap = 43
    if input_json['image_params']['overlap']:
        overlap = 70

    img_size_deg_lat = m_per_px_lat * deg_per_m_lat * (input_json['image_params']['size']-overlap)  # m/px * deg/m * px = deg
    img_size_deg_lon = m_per_px_lon * deg_per_m_lon * (input_json['image_params']['size']-overlap)  # m/px * deg/m * px = deg


    if data['grid_size'] == 1:
        return [data['center']]
    elif data['grid_size'] % 2 != 0 : # impar 3x3 5x5 7x7 ....
        n = data['grid_size']
        centers = []
        lats = []
        lons = []
        
        lat_0 = data['center']['lat'] + (n-1) * (img_size_deg_lat / 2)
        lon_0 = data['center']['lon'] - (n-1) * (img_size_deg_lon / 2)

        for a in range(n):
            lat = lat_0 - img_size_deg_lat * a
            lon = lon_0 + img_size_deg_lon * a
            lats.append(lat)
            lons.append(lon)

        for i in range(len(lats)):
            lat = lats[i]
            for j in range(len(lons)):
                lon = lons[j]
                centers.append({
                    'pos': str(i) + "-" + str(j),
                    'lat': lat,
                    'lon': lon,
                })

        return centers
    elif data['grid_size'] % 2 == 0: # par 2x2 4x4 6x6

        n = data['grid_size']

        centers = []
        lats = []
        lons = []
        
        lat_0 = data['center']['lat'] + (n-1) * ( img_size_deg_lat / 2 )
        lon_0 = data['center']['lon'] - (n-1) * ( img_size_deg_lon / 2 )

        for a in range(n):
            lat = lat_0 - img_size_deg_lat * a
            lon = lon_0 + img_size_deg_lon * a
            lats.append(lat)
            lons.append(lon)

        for i in range(len(lats)):
            lat = lats[i]
            for j in range(len(lons)):
                lon = lons[j]
                centers.append({
                    'pos': str(i) + "-" + str(j),
                    'lat': lat,
                    'lon': lon,
                })
        return centers
    else:
        return  []


In [ ]:

# Import the imput data from the JSON file
input_json = read_input()

for data in input_json['data']:
    grid = get_grid( data , input_json )
    for point in grid:
        data['center']['lat'] = point['lat']
        data['center']['lon'] = point['lon']    
        data['center']['pos'] = point['pos']
        img = get_image(data, input_json)
        save_image(data, input_json, img)


In [ ]:
import cv2
import numpy as np


for data in input_json['data']:
    image_ref_array = []
    images = []
    grid = get_grid(data , input_json)

    folder_name = get_folder_ref(data, input_json)

    for point in grid:
        data['center']['lat'] = point['lat']
        data['center']['lon'] = point['lon']    
        data['center']['pos'] = point['pos']

        image_name = get_image_name(data, input_json)
        image_ref = folder_name + image_name

        image_ref_array.append(image_ref)

    for ref in image_ref_array:
        aux = cv2.imread(ref)
        crop = input_json['image_params']['size']-43
        aux = aux[:crop,:]
        images.append(aux)

    horizontals_group = []
    counter = 0
    for i in range(data['grid_size']):
        line = []

        for j in range(data['grid_size']):
            line.append(images[counter])
            counter += 1

        line_img = cv2.hconcat(line)
        horizontals_group.append(line_img)
    
    complete_img = cv2.vconcat(horizontals_group)

    name =  data['name']
    grid_size = str(data['grid_size'])
    zoom = str(input_json['image_params']['zoom'])
    size = str(input_json['image_params']['size'])

    complete_img_ref = folder_name + name + '_' + grid_size + '_' + zoom + '_' + size + '.png'

    print(complete_img_ref)
    cv2.imwrite(complete_img_ref, complete_img)